In [1]:
from methods import *
from model import LogisticInstance
import numpy as np
import time

In [2]:
# simple 1D case
# e^{x-2} / (1 + e^{x-2}) (-x + 5) --> max
# theoretical answer: x_opt ~ 2.44
# with constraint x<=2 x_opt ~ 2

a = np.array([1])
b = -2
c = np.array([-1])
d = 5
F = np.array([[1]])
g = np.array([2])

issue = LogisticInstance(a, b, c, d, F, g)

In [3]:
def test(method_name, method, real_f_name, real_f, **kwargs):
    t = time.monotonic()
    x_opt = method(**kwargs)
    s = ' ' * (len(method_name) + 2)
    print('{}: x_opt = {}\n{}{} = {}\n{}time = {}s'.format(method_name, x_opt, s, 
                                                           real_f_name, real_f(x_opt), s, 
                                                           time.monotonic() - t))

In [4]:
# unconditional optimization

f = lambda x: -issue.log_expected_profit(x)
df = lambda x: -issue.dlog_expected_profit(x)



test('Gradient descent, conv. by argument, h_k = 0.1', GradientDescent(h=0.1), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Gradient descent, conv. by argument, h_k = 1/k', GradientDescent(h=lambda step: 1. / step), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Gradient descent, conv. by argument, h_k = 1/sqrt(k)', GradientDescent(h=lambda step: step ** -0.5), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Accelerated Nesterov gradient descent, h_k = ', AcceleratedNesterovGradientDescent(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

Gradient descent, conv. by argument, h_k = 0.1: x_opt = [2.44261703]
                                                expected_profit = 1.5571455818420734
                                                time = 0.014325677999295294s
Gradient descent, conv. by argument, h_k = 1/k: x_opt = [2.34713717]
                                                expected_profit = 1.554373820931351
                                                time = 0.13076300399916363s
Gradient descent, conv. by argument, h_k = 1/sqrt(k): x_opt = [2.44255797]
                                                      expected_profit = 1.5571455722441265
                                                      time = 0.005698984001355711s
Accelerated Nesterov gradient descent, h_k = : x_opt = [2.4427758]
                                               expected_profit = 1.5571455971164652
                                               time = 0.001390942998114042s


In [5]:
# conditional

# the previous issue (theoretically, x_opt=2)

g = lambda x: np.dot(issue.F , x) -issue.g
dg = lambda x: issue.F

test('Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)**0.7', PenaltyMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df,
     g=g, dg=dg)

Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)**0.7: x_opt = [2.00233491]
                                                                    expected_profit = 1.500582363040146
                                                                    time = 0.07475280599828693s


In [11]:
# simple 2d issue
# (x + y) / (1 + e^{x+y}) --> max
# x + y <= 1
#     y <= 0.5
# theoretically, x_opt = (0.5, 0.5)

a = np.array([-1, -1])
b = 0
c = np.array([1, 1])
d = 0
F = np.array([[1, 1],
              [0, 1]])
g = np.array([1, 0.5])

issue = LogisticInstance(a, b, c, d, F, g)

In [12]:
f = lambda x: -issue.log_expected_profit(x)
df = lambda x: -issue.dlog_expected_profit(x)
g = lambda x: np.dot(issue.F , x) -issue.g
dg = lambda x: issue.F

test('Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)**0.7', PenaltyMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.ones(issue.a.shape[0]) * 0.01, df=df,
     g=g, dg=dg)

Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)**0.7: x_opt = [0.50177973 0.50177973]
                                                                    expected_profit = 0.26919696224432765
                                                                    time = 0.0414184170003864s
